In [16]:
import pandas as pd

import sys
sys.path.append("src")
from merging_with_id import merging, create_id

In [3]:
# load datasets
datasets = {}
years = range(2016, 2024)
for year in years:
    print(f"EIE {year} Loading...")
    if int(year) >= 2019:
        file_name = f'Odata{year}File.csv'
    else:
        file_name = f'OpenData{year}.csv'
    try:           
        dataset = pd.read_csv(f"../../data_loader/{year}/{file_name}", sep=";", encoding='utf-8', dtype = str)
    except:
        dataset = pd.read_csv(f"../../data_loader/{year}/{file_name}", sep=";", encoding='Windows 1251', dtype = str)
    datasets.update({year:dataset})
    print("success")

EIE 2016 Loading...
success
EIE 2017 Loading...
success
EIE 2018 Loading...
success
EIE 2019 Loading...
success
EIE 2020 Loading...
success
EIE 2021 Loading...
success
EIE 2022 Loading...
success
EIE 2023 Loading...
success


In [20]:
for year, dataset in datasets.items():
    #lowercase all columns and add year as attribute
    dataset.columns = [col.lower() for col in dataset.columns]
    dataset['year'] = year

In [21]:
for year, dataset in datasets.items():
    print([col for col in dataset.columns if 'pt' in col])

['ukrptname', 'ukrptregname', 'ukrptareaname', 'ukrpttername', 'histptname', 'histptregname', 'histptareaname', 'histpttername', 'mathptname', 'mathptregname', 'mathptareaname', 'mathpttername', 'physptname', 'physptregname', 'physptareaname', 'physpttername', 'chemptname', 'chemptregname', 'chemptareaname', 'chempttername', 'bioptname', 'bioptregname', 'bioptareaname', 'biopttername', 'geoptname', 'geoptregname', 'geoptareaname', 'geopttername', 'engptname', 'engptregname', 'engptareaname', 'engpttername', 'frptname', 'frptregname', 'frptareaname', 'frpttername', 'deuptname', 'deuptregname', 'deuptareaname', 'deupttername', 'sptest', 'spteststatus', 'spptname', 'spptregname', 'spptareaname', 'sppttername', 'rusptname', 'rusptregname', 'rusptareaname', 'ruspttername']
['ukrptname', 'ukrptregname', 'ukrptareaname', 'ukrpttername', 'histptname', 'histptregname', 'histptareaname', 'histpttername', 'mathptname', 'mathptregname', 'mathptareaname', 'mathpttername', 'physptname', 'physptregna

In [48]:
test_centers_df = pd.DataFrame(columns=['ptname', 'ptregname', 'ptareaname', 'pttername', 'year'])

In [49]:
for year, dataset in datasets.items():
    prefixes = [col[:-6] for col in dataset.columns if 'ptname' in col]
    for prefix in prefixes:
        dataset_help = dataset.loc[:, [prefix + 'ptname', prefix + 'ptregname', prefix + 'ptareaname', prefix + 'pttername', 'year']]
        dataset_help.rename(columns={prefix + 'ptname': 'ptname', prefix + 'ptregname': 'ptregname', prefix + 'ptareaname': 'ptareaname', prefix + 'pttername': 'pttername'}, inplace=True)
        test_centers_df = pd.concat([test_centers_df, dataset_help], ignore_index=True)
        # print(year, prefix, dataset_help.dropna(how = 'all').shape[0])
    
test_centers_df

,ptname,ptregname,ptareaname,pttername,year
0,Мелітопольська загальноосвітня школа І-ІІІ сту...,Запорізька область,м.Мелітополь,м.Мелітополь,2016
1,"Красилівський навчально-виховний комплекс ""Гім...",Хмельницька область,Красилівський район,м.Красилів,2016
2,Першотравенська загальноосвітня школа І-ІІІ ст...,Дніпропетровська область,м.Першотравенськ,м.Першотравенськ,2016
3,Чернівецька загальноосвітня школа І-ІІІ ступен...,Чернівецька область,м.Чернівці. Шевченківський район міста,Шевченківський район міста,2016
4,Первомайська загальноосвітня школа I-III ступе...,Миколаївська область,м.Первомайськ,м.Первомайськ,2016
...,...,...,...,...,...
22924687,NaN,NaN,NaN,NaN,2021
22924688,NaN,NaN,NaN,NaN,2021
22924689,NaN,NaN,NaN,NaN,2021
22924690,NaN,NaN,NaN,NaN,2021


In [50]:
test_centers_df.columns

Index(['ptname', 'ptregname', 'ptareaname', 'pttername', 'year'], dtype='object')

In [51]:
test_centers_df.drop_duplicates(inplace=True)
# When the paricipant graduated in the previous years 
# the corresponding information about school (everything with 'eo') is NaN.
# We drop these rows.
test_centers_df.dropna(subset=test_centers_df.columns.difference(['year']), 
                      how = 'all', inplace=True)
test_centers_df.reset_index(drop=True, inplace=True)
test_centers_df

,ptname,ptregname,ptareaname,pttername,year
0,Мелітопольська загальноосвітня школа І-ІІІ сту...,Запорізька область,м.Мелітополь,м.Мелітополь,2016
1,"Красилівський навчально-виховний комплекс ""Гім...",Хмельницька область,Красилівський район,м.Красилів,2016
2,Першотравенська загальноосвітня школа І-ІІІ ст...,Дніпропетровська область,м.Першотравенськ,м.Першотравенськ,2016
3,Чернівецька загальноосвітня школа І-ІІІ ступен...,Чернівецька область,м.Чернівці. Шевченківський район міста,Шевченківський район міста,2016
4,Первомайська загальноосвітня школа I-III ступе...,Миколаївська область,м.Первомайськ,м.Первомайськ,2016
...,...,...,...,...,...
8860,Одеська загальноосвітня школа № 23 I-III ступе...,Одеська область,м.Одеса. Суворовський район міста,Суворовський район міста,2021
8861,Великодолинський заклад загальної середньої ос...,Одеська область,Овідіопольський район,смт Великодолинське,2021
8862,"Комунальний заклад освіти ""Спеціалізована школ...",Дніпропетровська область,м.Дніпро. Амур-Нижньодніпровський район міста,Амур-Нижньодніпровський район міста,2021
8863,"Комунальний заклад освіти ""Середня загальноосв...",Дніпропетровська область,м.Дніпро. Шевченківський район міста,Шевченківський район міста,2021


In [52]:
locations_df = pd.read_csv("location_data/locations.csv",  encoding='utf-8', dtype=str)
locations_df

,regname,areaname,tername,KOATUU_2020,KATOTTG_2023
0,Запорізька область,Мелітопольський район,с.Терпіння,2323085101,UA23080270010078454
1,Хмельницька область,Красилівський район,м.Красилів,6822710100,UA68040210010032567
2,Дніпропетровська область,Петропавлівський район,с.Дмитрівка,1223881501,UA12140170040016918
3,Чернівецька область,м.Чернівці,Шевченківський район міста,7310100000,UA73060610010033137
4,Миколаївська область,Врадіївський район,с.Кумарі,4822383001,UA48080050190079797
...,...,...,...,...,...
29214,США,м.Детройт,м.Детройт,0026010000,OC26010000000000000
29215,Велика Британія,м.Абериствіт,м.Абериствіт,0008030000,OC08030000000000000
29216,США,м.Нью-Йорк,м.Нью-Йорк,0026020000,OC26020000000000000
29217,США,м.Х'юстон,м.Х'юстон,0026040000,OC26040000000000000


In [53]:
locations_df.rename(columns={'regname':'ptregname', 'areaname':'ptareaname', 'tername':'pttername'}, inplace=True)
locations_df.columns

Index(['ptregname', 'ptareaname', 'pttername', 'KOATUU_2020', 'KATOTTG_2023'], dtype='object')

In [54]:
# add location codifier and aggregate by years
test_centers_df_loc = test_centers_df.merge(locations_df[['ptregname', 'ptareaname', 'pttername', 'KOATUU_2020']], on=['ptregname', 'ptareaname', 'pttername'], how='left')

test_centers_df_loc

,ptname,ptregname,ptareaname,pttername,year,KOATUU_2020
0,Мелітопольська загальноосвітня школа І-ІІІ сту...,Запорізька область,м.Мелітополь,м.Мелітополь,2016,2310700000
1,"Красилівський навчально-виховний комплекс ""Гім...",Хмельницька область,Красилівський район,м.Красилів,2016,6822710100
2,Першотравенська загальноосвітня школа І-ІІІ ст...,Дніпропетровська область,м.Першотравенськ,м.Першотравенськ,2016,1212600000
3,Чернівецька загальноосвітня школа І-ІІІ ступен...,Чернівецька область,м.Чернівці. Шевченківський район міста,Шевченківський район міста,2016,7310100000
4,Первомайська загальноосвітня школа I-III ступе...,Миколаївська область,м.Первомайськ,м.Первомайськ,2016,4810400000
...,...,...,...,...,...,...
8860,Одеська загальноосвітня школа № 23 I-III ступе...,Одеська область,м.Одеса. Суворовський район міста,Суворовський район міста,2021,5110137600
8861,Великодолинський заклад загальної середньої ос...,Одеська область,Овідіопольський район,смт Великодолинське,2021,5123755300
8862,"Комунальний заклад освіти ""Спеціалізована школ...",Дніпропетровська область,м.Дніпро. Амур-Нижньодніпровський район міста,Амур-Нижньодніпровський район міста,2021,1210136300
8863,"Комунальний заклад освіти ""Середня загальноосв...",Дніпропетровська область,м.Дніпро. Шевченківський район міста,Шевченківський район міста,2021,1210136600


In [55]:
test_centers_df_loc = test_centers_df_loc.loc[:, ['ptname', 'year', 'KOATUU_2020']]
test_centers_df_loc

,ptname,year,KOATUU_2020
0,Мелітопольська загальноосвітня школа І-ІІІ сту...,2016,2310700000
1,"Красилівський навчально-виховний комплекс ""Гім...",2016,6822710100
2,Першотравенська загальноосвітня школа І-ІІІ ст...,2016,1212600000
3,Чернівецька загальноосвітня школа І-ІІІ ступен...,2016,7310100000
4,Первомайська загальноосвітня школа I-III ступе...,2016,4810400000
...,...,...,...
8860,Одеська загальноосвітня школа № 23 I-III ступе...,2021,5110137600
8861,Великодолинський заклад загальної середньої ос...,2021,5123755300
8862,"Комунальний заклад освіти ""Спеціалізована школ...",2021,1210136300
8863,"Комунальний заклад освіти ""Середня загальноосв...",2021,1210136600


In [56]:
test_centers_df_loc[test_centers_df_loc.KOATUU_2020.isna()]

,ptname,year,KOATUU_2020


In [57]:
# add codifier for region
test_centers_df_loc['KOATUU_2020_reg'] = test_centers_df_loc['KOATUU_2020'].str[:2]
test_centers_df_loc

,ptname,year,KOATUU_2020,KOATUU_2020_reg
0,Мелітопольська загальноосвітня школа І-ІІІ сту...,2016,2310700000,23
1,"Красилівський навчально-виховний комплекс ""Гім...",2016,6822710100,68
2,Першотравенська загальноосвітня школа І-ІІІ ст...,2016,1212600000,12
3,Чернівецька загальноосвітня школа І-ІІІ ступен...,2016,7310100000,73
4,Первомайська загальноосвітня школа I-III ступе...,2016,4810400000,48
...,...,...,...,...
8860,Одеська загальноосвітня школа № 23 I-III ступе...,2021,5110137600,51
8861,Великодолинський заклад загальної середньої ос...,2021,5123755300,51
8862,"Комунальний заклад освіти ""Спеціалізована школ...",2021,1210136300,12
8863,"Комунальний заклад освіти ""Середня загальноосв...",2021,1210136600,12


In [58]:
# check whether there are duplicates by name and region
test_centers_df_loc[test_centers_df_loc.duplicated(subset=['ptname', 'year', 'KOATUU_2020_reg'], keep=False)]

,ptname,year,KOATUU_2020,KOATUU_2020_reg


In [59]:
test_centers_df_aggr = test_centers_df_loc.groupby(['ptname', 'KOATUU_2020', 'KOATUU_2020_reg'])['year'].agg(lambda x: list(x.unique())).reset_index()
test_centers_df_aggr

,ptname,KOATUU_2020,KOATUU_2020_reg,year
0,"""Запорізький національний університет"" Міністе...",2310136300,23,[2017]
1,"""Запорізький національний університет"" Міністе...",2310136300,23,[2018]
2,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2016]
3,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2017]
4,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2018]
...,...,...,...,...
3476,Ясінянська загальноосвітня школа І-ІІІ ступені...,2123656200,21,"[2016, 2017]"
3477,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,[2021]
3478,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,"[2017, 2018, 2019, 2020]"
3479,"комунальний заклад освіти ""Середня загальноосв...",1210137800,12,[2021]


In [60]:
#check whether len of location codifier is right
test_centers_df_aggr[test_centers_df_aggr.KOATUU_2020.str.len() != 10]

,ptname,KOATUU_2020,KOATUU_2020_reg,year


In [62]:
create_id(test_centers_df_aggr, 'ptname', 'id')

,ptname,KOATUU_2020,KOATUU_2020_reg,year,id
0,"""Запорізький національний університет"" Міністе...",2310136300,23,[2017],запорзькийнацональнийунверситетмнстерстваосвти...
1,"""Запорізький національний університет"" Міністе...",2310136300,23,[2018],запорзькийнацональнийунверситетмнстерстваосвти...
2,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2016],навчальновиховнийкомплексовруцькагмназяменандр...
3,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2017],навчальновиховнийкомплексовруцькагмназяменандр...
4,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2018],навчальновиховнийкомплексовруцькагмназяменандр...
...,...,...,...,...,...
3476,Ясінянська загальноосвітня школа І-ІІІ ступені...,2123656200,21,"[2016, 2017]",яснянськазагальноосвтняшкола3ступ2рахвськазака...
3477,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,[2021],загальноосвтняшкола93ступмзмалодеськаобл
3478,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,"[2017, 2018, 2019, 2020]",загальноосвтняшкола93ступмзмалодеськаобл
3479,"комунальний заклад освіти ""Середня загальноосв...",1210137800,12,[2021],середнязагальноосвтняшкола62


# MON data

## Full name

In [63]:
#load data
schools_data = pd.read_excel('./school_data/Schools_02.08.2023 16-56.xlsx', dtype = str)
schools_data

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,Код ЄДРПОУ,Статус,Тип закладу,Форма власності,Код КОАТУУ,Регіон,...,Телефон,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,(0800)330810,NaN,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)5264358,(044)5262002,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,"(044)234-50-55, (044)235-60-63",(044)234-50-55,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)259-31-60,(044)525-30-31,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)2584242,NaN,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,"Яблунівський навчально-виховний комплекс ""Зага...",144723,так,NaN,33329891,ліквідовано,навчально-виховний комплекс (об'єднання),Комунальна,7423886502,Чернігівська область,...,(046)422-43-40,NaN,solovejzanna8@gmail.com,NaN,Директор Новік Ольга Іванівна,ні,так,ні,ні,1
17331,"Ягіднянська філія опорного закладу ""Количівськ...",145351,так,NaN,43195202,ліквідовано,школа,Комунальна,7425582803,Чернігівська область,...,(0462)687733,NaN,yagidneschool@ukr.net,NaN,Завідувач філією Курилець Ніна Михайлівна,Філія,так,ні,ні,NaN
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,(04653)27571,NaN,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,(046)5924644,NaN,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90


In [64]:
schools_data.columns

Index(['Повна назва', 'Код в ЄДЕБО', 'ОУО підтвердив дані', 'Скорочена назва',
       'Код ЄДРПОУ', 'Статус', 'Тип закладу', 'Форма власності', 'Код КОАТУУ',
       'Регіон', 'Населений пункт', 'Місцезнаходження (юридична адреса)',
       'Головний заклад',
       'Найменування органу, до сфери управління якого належить заклад освіти',
       'Телефон', 'Факс', 'Електронна пошта', 'Веб-сайт', 'Керівник',
       'Опорний / Філія', 'Сільський', 'Гірський', 'Інтернат',
       'Ліцензовані обсяги'],
      dtype='object')

In [91]:
# create new with id
schools_data_id = schools_data.copy()
create_id(schools_data_id, 'Повна назва', 'id')

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,Код ЄДРПОУ,Статус,Тип закладу,Форма власності,Код КОАТУУ,Регіон,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,NaN,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN,приватнийзакладосвтикмдш2мки
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)5262002,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN,гмназя236мкив
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)234-50-55,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN,гмназя87мкив
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)525-30-31,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN,гмназя150мкив
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,NaN,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN,гмназя179мкив
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,"Яблунівський навчально-виховний комплекс ""Зага...",144723,так,NaN,33329891,ліквідовано,навчально-виховний комплекс (об'єднання),Комунальна,7423886502,Чернігівська область,...,NaN,solovejzanna8@gmail.com,NaN,Директор Новік Ольга Іванівна,ні,так,ні,ні,1,яблунвськийнавчальновиховнийкомплексзагальноос...
17331,"Ягіднянська філія опорного закладу ""Количівськ...",145351,так,NaN,43195202,ліквідовано,школа,Комунальна,7425582803,Чернігівська область,...,NaN,yagidneschool@ukr.net,NaN,Завідувач філією Курилець Ніна Михайлівна,Філія,так,ні,ні,NaN,ягднянськафляопорногозакладуколичвськазагально...
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,NaN,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1,ядутинськазагальноосвтняшкола3ступборзнянськач...
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,NaN,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90,янжулвськафля2ступсеменвськагмназ2семенвськаче...


In [92]:
schools_data_id.rename(columns={'Код КОАТУУ':'KOATUU_2020', 'Код ЄДРПОУ':'EDRPOU'}, inplace=True)
schools_data_id.columns

Index(['Повна назва', 'Код в ЄДЕБО', 'ОУО підтвердив дані', 'Скорочена назва',
       'EDRPOU', 'Статус', 'Тип закладу', 'Форма власності', 'KOATUU_2020',
       'Регіон', 'Населений пункт', 'Місцезнаходження (юридична адреса)',
       'Головний заклад',
       'Найменування органу, до сфери управління якого належить заклад освіти',
       'Телефон', 'Факс', 'Електронна пошта', 'Веб-сайт', 'Керівник',
       'Опорний / Філія', 'Сільський', 'Гірський', 'Інтернат',
       'Ліцензовані обсяги', 'id'],
      dtype='object')

In [93]:
schools_data_id[schools_data_id.KOATUU_2020.str.len() != 10]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id


In [96]:
schools_data_id[(schools_data_id.EDRPOU.str.len() < 8)&(schools_data_id.EDRPOU.str.len().notna())]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id


In [95]:
schools_data_id['EDRPOU']=schools_data_id['EDRPOU'].apply(lambda x: '0'+x if (isinstance(x, str) and 0<len(x)<8) else x)

In [72]:
schools_data_id = schools_data_id[schools_data_id.EDRPOU.str.len() == 8]

In [73]:
schools_data_id.loc[:,'KOATUU_2020_reg'] = schools_data_id.loc[:,'KOATUU_2020'].str[:2]
schools_data_id

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_54084/1999010892.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schools_data_id.loc[:,'KOATUU_2020_reg'] = schools_data_id.loc[:,'KOATUU_2020'].str[:2]


,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN,приватнийзакладосвтикмдш2мки,80
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN,гмназя236мкив,80
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN,гмназя87мкив,80
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN,гмназя150мкив,80
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN,гмназя179мкив,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,"Яблунівський навчально-виховний комплекс ""Зага...",144723,так,NaN,33329891,ліквідовано,навчально-виховний комплекс (об'єднання),Комунальна,7423886502,Чернігівська область,...,solovejzanna8@gmail.com,NaN,Директор Новік Ольга Іванівна,ні,так,ні,ні,1,яблунвськийнавчальновиховнийкомплексзагальноос...,74
17331,"Ягіднянська філія опорного закладу ""Количівськ...",145351,так,NaN,43195202,ліквідовано,школа,Комунальна,7425582803,Чернігівська область,...,yagidneschool@ukr.net,NaN,Завідувач філією Курилець Ніна Михайлівна,Філія,так,ні,ні,NaN,ягднянськафляопорногозакладуколичвськазагально...,74
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1,ядутинськазагальноосвтняшкола3ступборзнянськач...,74
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90,янжулвськафля2ступсеменвськагмназ2семенвськаче...,74


In [74]:
schools_data_id[schools_data_id.duplicated(subset=['id', 'KOATUU_2020_reg', 'EDRPOU'], keep=False)]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
1611,Велимченський ліцей Велимченської сільської ра...,176569,так,Велимченський ліцей,21747814,ліквідовано,ліцей,Комунальна,0724280401,Волинська область,...,NaN,NaN,Директор Павлович Тетяна Володимирівна,ні,ні,ні,ні,NaN,велимченськийлцейвелимченськаволинськаобл,07
1612,Велимченський ліцей Велимченської сільської ра...,143856,так,Велимченський ліцей,21747814,працює,ліцей,Комунальна,0724280401,Волинська область,...,velimche__shcola@ukr.net,http://velimcheshkola.at.ua/,Павлович Тетяна Володимирівна,ні,так,ні,ні,NaN,велимченськийлцейвелимченськаволинськаобл,07


In [75]:
schools_data_id = schools_data_id.drop_duplicates(subset=['id', 'KOATUU_2020_reg', 'EDRPOU'], keep='last')

In [76]:
schools_data_id[schools_data_id.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
91,"Приватний заклад освіти ""УОЛД СКУЛ КАМПУС А""",151623,так,"ПЗО ""УОЛД СКУЛ КАМПУС А""",42180923,працює,школа,Приватна,8036300000,КИЇВ,...,campus.a@worldschool.com.ua,NaN,Директор Бадаєва Інна Миколаївна,ні,ні,ні,ні,NaN,приватнийзакладосвтиуолдскулкампус,80
254,Технічний ліцей м. Києва,145237,так,Технічний ліцей м. Києва,22876023,працює,ліцей,Комунальна,8036600000,КИЇВ,...,dtl-licej@ukr.ua,http://tl.kyiv.ua/,Директор Андрієнко Анатолій Михайлович,ні,ні,ні,ні,NaN,технчнийлцеймкив,80
268,"Приватний заклад освіти ""УОЛД СКУЛ КАМПУС Б""",176647,так,"ПЗО ""УОЛД СКУЛ КАМПУС Б""",43594721,працює,школа,Приватна,8000000000,КИЇВ,...,NaN,NaN,Директор Давидова Наталія Леонідівна,ні,ні,ні,ні,NaN,приватнийзакладосвтиуолдскулкампус,80
584,Технічний ліцей Шевченківського району м. Києва,149078,так,Технічний ліцей,22881946,працює,ліцей,Комунальна,8039100000,КИЇВ,...,texlyceum@ukr.net,techlyceum.kiev.ua,Директор Ілюшина Олена Олександрівна,ні,ні,ні,ні,844,технчнийлцеймкив,80
4728,Данилівський заклад загальної середньої освіти...,176502,так,Данилівський НВК І-ІІІ ступенів,40760974,працює,навчально-виховний комплекс (об'єднання),Комунальна,2125382601,Закарпатська область,...,danylovo@ukr.net,NaN,Директор Гайович Тетяна Василівна,ні,так,ні,ні,171,данилвськийзакладзагальнасередньаосвти3ступхус...,21
4729,Данилівський заклад загальної середньої освіти...,148256,так,ДАНИЛІВСЬКИЙ ЗЗСО І-ІІІ СТУПЕНІВ,34440724,реорганізовано,навчально-виховний комплекс (об'єднання),Комунальна,2125382601,Закарпатська область,...,danylovo@ukr.net,NaN,Директор Гайович Тетяна Василівна,ні,так,ні,ні,NaN,данилвськийзакладзагальнасередньаосвти3ступхус...,21
6818,Гоголівський ліцей Великодимерської селищної р...,146689,так,Гоголівський ліцей,25299425,працює,ліцей,Комунальна,3221281601,Київська область,...,gogoliv.school@gmail.com,http://gogolivska-zosh.kiev.sch.in.ua/,Директор Дяченко Оксана Іванівна,Опорний заклад,так,ні,ні,NaN,гоголвськийлцейвеликодимерськакивськаобл,32
6819,Гоголівський ліцей Великодимерської селищної р...,147130,так,філія Гоголівського ліцею,25299342,працює,ліцей,Комунальна,3221281601,Київська область,...,gogoliv-school2@ukr.net,http://gogoliv-school2.kiev.sch.in.ua,Завідувач філією Мишкоріз Любов Миколаївна,Філія,так,ні,ні,NaN,гоголвськийлцейвеликодимерськакивськаобл,32
11534,"Комунальний заклад ""Розсошенська гімназія Щерб...",139999,так,Розсошенська гімназія,21071866,працює,гімназія,Комунальна,5324087705,Полтавська область,...,rzsgymn@gmail.com,http://rzsschool.org.ua/,Директор Яловець Євген Олександрович,ні,так,ні,ні,834,розсошенськагмназящербанвськаполтавськаобл,53
11535,"Комунальний заклад ""Розсошенська гімназія Щерб...",151527,так,Розсошенська гімназія,52852882,ліквідовано,гімназія,Комунальна,5324087705,Полтавська область,...,rzsgymn@gmail.com,http://rzsschool.ucoz.ua/,Директор Яловець Євген Олександрович,ні,так,ні,ні,NaN,розсошенськагмназящербанвськаполтавськаобл,53


In [77]:
schools_data_id = schools_data_id[~schools_data_id.EDRPOU.isin(['34440724', '52852882', '58826264', 
                                                                '33329415', '26271178', '27646632', 
                                                                '04393999', '24623237', '25649174',
                                                                '25649172'])]

In [78]:
schools_data_id.drop(schools_data_id[schools_data_id.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)].index.values.tolist(), inplace=True)

In [79]:
test_centers_mon= test_centers_df_aggr.merge(schools_data_id[['KOATUU_2020_reg', 'EDRPOU', 'id']], on=['id', 'KOATUU_2020_reg'], how='left')
test_centers_mon.head()

,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU
0,"""Запорізький національний університет"" Міністе...",2310136300,23,[2017],запорзькийнацональнийунверситетмнстерстваосвти...,NaN
1,"""Запорізький національний університет"" Міністе...",2310136300,23,[2018],запорзькийнацональнийунверситетмнстерстваосвти...,NaN
2,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2016],навчальновиховнийкомплексовруцькагмназяменандр...,NaN
3,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2017],навчальновиховнийкомплексовруцькагмназяменандр...,NaN
4,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2018],навчальновиховнийкомплексовруцькагмназяменандр...,NaN


In [81]:
test_centers_mon.loc[test_centers_mon['EDRPOU'].isna(), 'EDRPOU'] = ''

In [82]:
test_centers_mon[test_centers_mon.EDRPOU != '']

,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU
5,"""Навчально-виховний комплекс ""Школа І-ІІ ступе...",8039100000,80,"[2018, 2021]",навчальновиховнийкомплексшкола2ступлцей38мвммо...,22881780
6,"""Теребовлянська загальноосвітня школа І-ІІІ ст...",6125010100,61,[2021],теребовлянськазагальноосвтняшкола3ступ2щознахо...,24618748
55,Ірпінський ліцей №3 Ірпінської міської ради Ки...,3210900000,32,"[2020, 2021]",рпнськийлцей3рпнськакивськаобл,25667797
60,Іршавський заклад загальної середньої освіти І...,2121910100,21,"[2019, 2020, 2021]",ршавськийзакладзагальнасередньаосвти3ступ2ршав...,22099980
61,Іршавський ліцей Іршавської міської ради Закар...,2121910100,21,"[2019, 2020, 2021]",ршавськийлцейршавськазакарпатськаобл,33186757
...,...,...,...,...,...,...
3443,Школа І-ІІІ ступенів №90 Печерського району мі...,8038200000,80,"[2016, 2017, 2018, 2019, 2020, 2021]",школа3ступ90мкив,32492367
3468,Яворівська гімназія імені Осипа Маковея Яворів...,4625810100,46,[2021],яворвськагмназяменосипамаковеяяворвськальввськ...,22398598
3470,Яворівський заклад загальної середньої освіти ...,4625810100,46,[2021],яворвськийзакладзагальнасередньаосвти3ступ3мен...,22398718
3475,Яремчанський ліцей №1 Яремчанської міської рад...,2611000000,26,[2021],яремчанськийлцей1яремчанськаванофранквськаобл,20556113


## Short name

In [83]:
schools_data_id_short = schools_data_id[schools_data_id['Скорочена назва'].notna()]

In [84]:
create_id(schools_data_id_short, 'Скорочена назва', 'id')

/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/ZNO-Dataset/notebooks/tables_creation/src/merging_with_id.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['id'] = dataset['id'].apply(del_rayon)
/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/ZNO-Dataset/notebooks/tables_creation/src/merging_with_id.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['id'] = dataset['id'].apply(lambda s: s if (len(s) and s[-1].isdigit()) else s[:-1]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN,приватнийзакладосвтикмдш2мки,80
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN,гмназя236,80
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN,гмназя87,80
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN,гмназя150,80
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN,гмназя179мкив,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,"комунальний заклад ""Яблунівський ліцей"" Черніг...",139767,так,"КЗ ""Яблунівський ліцей""",05266240,працює,ліцей,Комунальна,7424189701,Чернігівська область,...,Yablunovka@ukr.net,NaN,Директор Шаповаленко Неоніла Олександрівна,Опорний заклад,так,ні,так,170,яблунвськийлце,74
17329,Яблунівська загальноосвітня школа I-III ступен...,144175,так,Яблунівська ЗОШ І-ІІІ ступенів,33208204,працює,школа,Комунальна,7424189701,Чернігівська область,...,yablunivkask@ukr.net,NaN,Директор Колесник Анатолій Васильович,ні,так,ні,ні,320,яблунвськазагальноосвтняшкола3ступен,74
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1,ядутинськазагальноосвтняшкола3ст,74
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90,янжулвськафля2ст,74


In [85]:
schools_data_id_short[schools_data_id_short.duplicated(subset=['id', 'KOATUU_2020_reg', 'EDRPOU'], keep=False)]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg


In [86]:
schools_data_id_short =schools_data_id_short.drop_duplicates(subset=['id', 'EDRPOU', 'KOATUU_2020_reg'], keep='first') 

In [87]:
schools_data_id_short[schools_data_id_short.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
28,Спеціалізована школа І-ІІІ ступенів з поглибле...,148398,так,спеціалізована школа,22877005,працює,спеціалізована школа,Комунальна,8036100000,КИЇВ,...,sch15@i.ua,school15.kyiv.ua,Директор Литвин Валерій Олексійович,ні,ні,ні,ні,NaN,спецалзованашкол,80
496,ліцей №142 м. Києва,147801,так,ліцей,21622214,працює,ліцей,Комунальна,8038900000,КИЇВ,...,l142@ukr.net,NaN,Директор Сільвестрова Ірина Анатоліївна,ні,ні,ні,ні,600,лце,80
547,юридичний ліцей імені Ярослава Кондратьєва Нац...,147605,так,ліцей,35008569,працює,ліцей-інтернат,Державна,8038900000,КИЇВ,...,licei@naiau.kiev.ua,licey.naiau.kiev.ua,Ректор Чернєй Володимир Васильович,ні,ні,ні,ні,200,лце,80
572,Спеціалізована школа І-ІІІ ступенів №28 з погл...,149079,так,спеціалізована школа,22880792,працює,спеціалізована школа,Комунальна,8039100000,КИЇВ,...,school28kiev@ukr.net,sites.google.com/view/sch28,Директор Охріменко Наталія Миколаївна,ні,ні,ні,ні,1176,спецалзованашкол,80
652,Обласний науковий ліцей-інтернат Комунального ...,143380,так,Обласний науковий ліцей-інтернат,36364955,працює,науковий ліцей-інтернат,Комунальна,0510100000,Вінницька область,...,vinlitsey@ukr.net,https://www.vgpk.com.ua,Директор Войцехівський Костянтин Францович,ні,ні,ні,так,720,обласнийнауковийлцейнтерна,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15586,Рудська гімназія Смотрицької селищної ради Кам...,141431,так,Рудська гімназія,22777640,працює,гімназія,Комунальна,6821887701,Хмельницька область,...,rud.zosh2015@ukr.net,http://www.rudka-school.at.ua,Директор Середа Валентина Олексіївна,ні,так,ні,ні,NaN,рудськагмназ,68
16194,Рижавська філія Ладижинського ліцею Ладижинськ...,134934,так,Рижавська філія,24352264,працює,школа,Комунальна,7124387301,Черкаська область,...,rugavkaschool@gmail.com,NaN,Завідувач філією Квашук Оксана Миколаївна,Філія,ні,ні,ні,1,рижавськафл,71
16195,Рижавська філія Ладижинської загальноосвітньої...,151388,так,Рижавська філія,42469234,працює,школа,Комунальна,7124387301,Черкаська область,...,rugavkaschool@gmail.com,NaN,Завідувач філією Квашук Оксана Миколаївна,ні,так,ні,ні,1,рижавськафл,71
16551,Кам'янський ліцей Кам'янської сільської ради Ч...,142265,так,Кам'янський ліцей,21440097,працює,ліцей,Комунальна,7324584001,Чернівецька область,...,Kamjana-shkola@meta.ua,NaN,Директор Кожелянко Світлана Михайлівна,ні,так,ні,ні,550,камянськийлце,73


In [88]:
schools_data_id_short.drop(schools_data_id_short[schools_data_id_short.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)].index.values.tolist(), inplace=True)

In [90]:
test_centers_mon_final= merging(test_centers_mon, schools_data_id_short[['KOATUU_2020_reg', 'EDRPOU', 'id']], ['id', 'KOATUU_2020_reg'])
test_centers_mon_final

Percentage matched: 27.0 %


,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU
0,"""Запорізький національний університет"" Міністе...",2310136300,23,[2017],запорзькийнацональнийунверситетмнстерстваосвти...,
1,"""Запорізький національний університет"" Міністе...",2310136300,23,[2018],запорзькийнацональнийунверситетмнстерстваосвти...,
2,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2016],навчальновиховнийкомплексовруцькагмназяменандр...,
3,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2017],навчальновиховнийкомплексовруцькагмназяменандр...,
4,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2018],навчальновиховнийкомплексовруцькагмназяменандр...,
...,...,...,...,...,...,...
3476,Ясінянська загальноосвітня школа І-ІІІ ступені...,2123656200,21,"[2016, 2017]",яснянськазагальноосвтняшкола3ступ2рахвськазака...,
3477,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,[2021],загальноосвтняшкола93ступмзмалодеськаобл,
3478,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,"[2017, 2018, 2019, 2020]",загальноосвтняшкола93ступмзмалодеськаобл,
3479,"комунальний заклад освіти ""Середня загальноосв...",1210137800,12,[2021],середнязагальноосвтняшкола62,


# Universities

In [97]:
univ_data = pd.read_excel('./school_data/zakladi-vishchoyi-osviti-09.xlsx', dtype = str)
univ_data

,Назва закладу освіти,Код,Код головного закладу,Коротка назва,Назва закладу освіти (англ.),Рік заснування,Ідентифікаційний код,Вид закладу освіти,Тип закладу освіти,Форма власності,...,Поштовий індекс (юридична),Код КОАТУУ (юридична),Регіон (юридична),Населений пункт (юридична),Адреса (юридична),Телефон / факс,Електронна пошта,Веб-сайт,Посада керівника,"Прізвище, ім'я, по батькові керівника"
0,"Академія праці, соціальних відносин і туризму",53,NaN,АПСВТ,"Academy of Labour, Social Relations and Tourism",1993,04641405,Заклад вищої освіти,Академія,Приватна,...,03680,8000000000,КИЇВ,Київ,"вул. Велика Окружна дорога, 3",+38(044) 522-49-40,info@socosvita.kiev.ua,http://www. socosvita.kiev.ua,Ректор,Буяшенко Вікторія Василівна
1,Бізнес-коледж Приватного вищого навчального за...,3555,228,БК ПВНЗ «Європейський університет»,Business College of Private Higher Educational...,2002,40487422,Заклад вищої освіти,Коледж,Приватна,...,03115,8000000000,КИЇВ,Київ,"бульв. Академіка Вернадського, 16-В",+380(44)-432-00-58,bkeu@kyiv.e-u.in.ua,www.e-u.in.ua,директор,Сильницький Юрій Вікторович
2,"Вищий навчальний заклад ""Відкритий міжнародний...",217,NaN,"Університет ""Україна""",Higher Education Institution 'Open Internation...,1998,30373644,Заклад вищої освіти,Університет,Приватна,...,04071,8038500000,КИЇВ,Київ,"вул. Хорива, 1Г",(044) 4287410; 4252322; 4225019; 4252926,viddilorgrobotu@ukr.net,www.vmurol.com.ua,президент,Таланчук Петро Михайлович
3,"Вищий навчальний заклад Інститут дизайну, архі...",1319,NaN,"Вищий навчальний заклад ""Інститут дизайну, арх...",Higher educational institution “Institute of D...,2000,32594570,Заклад вищої освіти,Інститут,Приватна,...,03039,8038500000,КИЇВ,Київ,"вул. Фрометівська, 2",+380(44)-496-97-60,fb.mail.ir@gmail.com,www.ir.edu.ua,В. о. директора,Бойко Світлана Станіславівна
4,"Вищий навчальний заклад ""Київська Академія пер...",865,NaN,"ВНЗ ""КАПМ""","Higher educational institution ""Kyiv Academy o...",2002,26199252,Заклад вищої освіти,Академія,Приватна,...,03067,8038900000,КИЇВ,Київ,"вул. Виборзька, 86",0444555672,kapm@ukr.net,www.kapi.com.ua,ректор,Лубянська Світлана Павлівна
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,Відокремлений підрозділ національного універси...,2839,7,.,NaN,2000,35139007,Заклад вищої освіти,Відокремлений підрозділ,Державна,...,17400,7420610100,Чернігівська область,Бобровиця,"вул. Чернігівська, 19",NaN,NaN,NaN,NaN,NaN
1416,Борзнянський державний сільськогосподарський т...,390,NaN,БДСГТ,State agricultural technical school of Borzna,1999,00729178,Заклад вищої освіти,Технікум (училище),Державна,...,16400,7420810100,Чернігівська область,Борзна,"Олекси Десняка, 23",21106,koba_22@ukr.net,bdsgt.at.ua,Виконуючий обов`язки директора,Линник Віталій Володимирович
1417,Козелецький коледж ветеринарної медицини Білоц...,990,362,ККВМ БНАУ,"Kozelets College of Veterinary Medicine, Bila ...",1925,00729209,Заклад вищої освіти,Коледж,Державна,...,17000,7422055100,Чернігівська область,Козелець,"вул. Сім'ї Розумовських, 41","(046)462-12-00, (046)464-15-96, (046)462-14-83",techvet2@gmail.com,http://www.techvet.com.ua/,Директор коледжу,Редька Алла Іванівна
1418,Остерський коледж будівництва та дизайну,925,NaN,ОКБД,Oster college of civil engineering and designing,1945,01355708,Заклад вищої освіти,Коледж,Державна,...,17044,7422010400,Чернігівська область,Остер,"вул. 8 Березня, 7-А",(046) 464-32-85,oster_budtex@cg.ukrtel.net,http://okbd.16mb.com,Директор,Бардаченко Тетяна Миколаївна


In [98]:
univ_data_id = univ_data

In [99]:
create_id(univ_data_id, 'Назва закладу освіти', 'id')

,Назва закладу освіти,Код,Код головного закладу,Коротка назва,Назва закладу освіти (англ.),Рік заснування,Ідентифікаційний код,Вид закладу освіти,Тип закладу освіти,Форма власності,...,Код КОАТУУ (юридична),Регіон (юридична),Населений пункт (юридична),Адреса (юридична),Телефон / факс,Електронна пошта,Веб-сайт,Посада керівника,"Прізвище, ім'я, по батькові керівника",id
0,"Академія праці, соціальних відносин і туризму",53,NaN,АПСВТ,"Academy of Labour, Social Relations and Tourism",1993,04641405,Заклад вищої освіти,Академія,Приватна,...,8000000000,КИЇВ,Київ,"вул. Велика Окружна дорога, 3",+38(044) 522-49-40,info@socosvita.kiev.ua,http://www. socosvita.kiev.ua,Ректор,Буяшенко Вікторія Василівна,академяпрацсоцальнихвдносинтуризм
1,Бізнес-коледж Приватного вищого навчального за...,3555,228,БК ПВНЗ «Європейський університет»,Business College of Private Higher Educational...,2002,40487422,Заклад вищої освіти,Коледж,Приватна,...,8000000000,КИЇВ,Київ,"бульв. Академіка Вернадського, 16-В",+380(44)-432-00-58,bkeu@kyiv.e-u.in.ua,www.e-u.in.ua,директор,Сильницький Юрій Вікторович,бзнесколеджприватноговищогонавчальногозакладув...
2,"Вищий навчальний заклад ""Відкритий міжнародний...",217,NaN,"Університет ""Україна""",Higher Education Institution 'Open Internation...,1998,30373644,Заклад вищої освіти,Університет,Приватна,...,8038500000,КИЇВ,Київ,"вул. Хорива, 1Г",(044) 4287410; 4252322; 4225019; 4252926,viddilorgrobotu@ukr.net,www.vmurol.com.ua,президент,Таланчук Петро Михайлович,вищийнавчальнийзакладвдкритиймжнароднийунверси...
3,"Вищий навчальний заклад Інститут дизайну, архі...",1319,NaN,"Вищий навчальний заклад ""Інститут дизайну, арх...",Higher educational institution “Institute of D...,2000,32594570,Заклад вищої освіти,Інститут,Приватна,...,8038500000,КИЇВ,Київ,"вул. Фрометівська, 2",+380(44)-496-97-60,fb.mail.ir@gmail.com,www.ir.edu.ua,В. о. директора,Бойко Світлана Станіславівна,вищийнавчальнийзакладнститутдизайнуархтектурит...
4,"Вищий навчальний заклад ""Київська Академія пер...",865,NaN,"ВНЗ ""КАПМ""","Higher educational institution ""Kyiv Academy o...",2002,26199252,Заклад вищої освіти,Академія,Приватна,...,8038900000,КИЇВ,Київ,"вул. Виборзька, 86",0444555672,kapm@ukr.net,www.kapi.com.ua,ректор,Лубянська Світлана Павлівна,вищийнавчальнийзакладкивськаакадемяперукарсько...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,Відокремлений підрозділ національного універси...,2839,7,.,NaN,2000,35139007,Заклад вищої освіти,Відокремлений підрозділ,Державна,...,7420610100,Чернігівська область,Бобровиця,"вул. Чернігівська, 19",NaN,NaN,NaN,NaN,NaN,вдокремленийпдроздлнацональногоунверситетуборе...
1416,Борзнянський державний сільськогосподарський т...,390,NaN,БДСГТ,State agricultural technical school of Borzna,1999,00729178,Заклад вищої освіти,Технікум (училище),Державна,...,7420810100,Чернігівська область,Борзна,"Олекси Десняка, 23",21106,koba_22@ukr.net,bdsgt.at.ua,Виконуючий обов`язки директора,Линник Віталій Володимирович,борзнянськийдержавнийогосподарськийтехнку
1417,Козелецький коледж ветеринарної медицини Білоц...,990,362,ККВМ БНАУ,"Kozelets College of Veterinary Medicine, Bila ...",1925,00729209,Заклад вищої освіти,Коледж,Державна,...,7422055100,Чернігівська область,Козелець,"вул. Сім'ї Розумовських, 41","(046)462-12-00, (046)464-15-96, (046)462-14-83",techvet2@gmail.com,http://www.techvet.com.ua/,Директор коледжу,Редька Алла Іванівна,козелецькийколеджветеринарнамедициниблоцерквсь...
1418,Остерський коледж будівництва та дизайну,925,NaN,ОКБД,Oster college of civil engineering and designing,1945,01355708,Заклад вищої освіти,Коледж,Державна,...,7422010400,Чернігівська область,Остер,"вул. 8 Березня, 7-А",(046) 464-32-85,oster_budtex@cg.ukrtel.net,http://okbd.16mb.com,Директор,Бардаченко Тетяна Миколаївна,остерськийколеджбудвництватадизайн


In [100]:
univ_data_id.columns

Index(['Назва закладу освіти', 'Код', 'Код головного закладу', 'Коротка назва',
       'Назва закладу освіти (англ.)', 'Рік заснування',
       'Ідентифікаційний код', 'Вид закладу освіти', 'Тип закладу освіти',
       'Форма власності',
       'Найменування органу, до сфери управління якого належить заклад освіти',
       'Поштовий індекс (фактична)', 'Код КОАТУУ (фактична)',
       'Регіон (фактична)', 'Населений пункт (фактична)', 'Адреса (фактична)',
       'Поштовий індекс (юридична)', 'Код КОАТУУ (юридична)',
       'Регіон (юридична)', 'Населений пункт (юридична)', 'Адреса (юридична)',
       'Телефон / факс', 'Електронна пошта', 'Веб-сайт', 'Посада керівника',
       'Прізвище, ім'я, по батькові керівника', 'id'],
      dtype='object')

In [101]:
univ_data_id.rename(columns={'Код КОАТУУ (юридична)':'KOATUU_2020', 'Ідентифікаційний код':'EDRPOU'}, inplace=True)

In [102]:
univ_data_id = univ_data_id[univ_data_id.KOATUU_2020.str.len() == 10]

In [103]:
univ_data_id['KOATUU_2020_reg'] = univ_data_id['KOATUU_2020'].str[:2]
univ_data_id

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_54084/3478306073.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  univ_data_id['KOATUU_2020_reg'] = univ_data_id['KOATUU_2020'].str[:2]


,Назва закладу освіти,Код,Код головного закладу,Коротка назва,Назва закладу освіти (англ.),Рік заснування,EDRPOU,Вид закладу освіти,Тип закладу освіти,Форма власності,...,Регіон (юридична),Населений пункт (юридична),Адреса (юридична),Телефон / факс,Електронна пошта,Веб-сайт,Посада керівника,"Прізвище, ім'я, по батькові керівника",id,KOATUU_2020_reg
0,"Академія праці, соціальних відносин і туризму",53,NaN,АПСВТ,"Academy of Labour, Social Relations and Tourism",1993,04641405,Заклад вищої освіти,Академія,Приватна,...,КИЇВ,Київ,"вул. Велика Окружна дорога, 3",+38(044) 522-49-40,info@socosvita.kiev.ua,http://www. socosvita.kiev.ua,Ректор,Буяшенко Вікторія Василівна,академяпрацсоцальнихвдносинтуризм,80
1,Бізнес-коледж Приватного вищого навчального за...,3555,228,БК ПВНЗ «Європейський університет»,Business College of Private Higher Educational...,2002,40487422,Заклад вищої освіти,Коледж,Приватна,...,КИЇВ,Київ,"бульв. Академіка Вернадського, 16-В",+380(44)-432-00-58,bkeu@kyiv.e-u.in.ua,www.e-u.in.ua,директор,Сильницький Юрій Вікторович,бзнесколеджприватноговищогонавчальногозакладув...,80
2,"Вищий навчальний заклад ""Відкритий міжнародний...",217,NaN,"Університет ""Україна""",Higher Education Institution 'Open Internation...,1998,30373644,Заклад вищої освіти,Університет,Приватна,...,КИЇВ,Київ,"вул. Хорива, 1Г",(044) 4287410; 4252322; 4225019; 4252926,viddilorgrobotu@ukr.net,www.vmurol.com.ua,президент,Таланчук Петро Михайлович,вищийнавчальнийзакладвдкритиймжнароднийунверси...,80
3,"Вищий навчальний заклад Інститут дизайну, архі...",1319,NaN,"Вищий навчальний заклад ""Інститут дизайну, арх...",Higher educational institution “Institute of D...,2000,32594570,Заклад вищої освіти,Інститут,Приватна,...,КИЇВ,Київ,"вул. Фрометівська, 2",+380(44)-496-97-60,fb.mail.ir@gmail.com,www.ir.edu.ua,В. о. директора,Бойко Світлана Станіславівна,вищийнавчальнийзакладнститутдизайнуархтектурит...,80
4,"Вищий навчальний заклад ""Київська Академія пер...",865,NaN,"ВНЗ ""КАПМ""","Higher educational institution ""Kyiv Academy o...",2002,26199252,Заклад вищої освіти,Академія,Приватна,...,КИЇВ,Київ,"вул. Виборзька, 86",0444555672,kapm@ukr.net,www.kapi.com.ua,ректор,Лубянська Світлана Павлівна,вищийнавчальнийзакладкивськаакадемяперукарсько...,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,Відокремлений підрозділ національного універси...,2839,7,.,NaN,2000,35139007,Заклад вищої освіти,Відокремлений підрозділ,Державна,...,Чернігівська область,Бобровиця,"вул. Чернігівська, 19",NaN,NaN,NaN,NaN,NaN,вдокремленийпдроздлнацональногоунверситетуборе...,74
1416,Борзнянський державний сільськогосподарський т...,390,NaN,БДСГТ,State agricultural technical school of Borzna,1999,00729178,Заклад вищої освіти,Технікум (училище),Державна,...,Чернігівська область,Борзна,"Олекси Десняка, 23",21106,koba_22@ukr.net,bdsgt.at.ua,Виконуючий обов`язки директора,Линник Віталій Володимирович,борзнянськийдержавнийогосподарськийтехнку,74
1417,Козелецький коледж ветеринарної медицини Білоц...,990,362,ККВМ БНАУ,"Kozelets College of Veterinary Medicine, Bila ...",1925,00729209,Заклад вищої освіти,Коледж,Державна,...,Чернігівська область,Козелець,"вул. Сім'ї Розумовських, 41","(046)462-12-00, (046)464-15-96, (046)462-14-83",techvet2@gmail.com,http://www.techvet.com.ua/,Директор коледжу,Редька Алла Іванівна,козелецькийколеджветеринарнамедициниблоцерквсь...,74
1418,Остерський коледж будівництва та дизайну,925,NaN,ОКБД,Oster college of civil engineering and designing,1945,01355708,Заклад вищої освіти,Коледж,Державна,...,Чернігівська область,Остер,"вул. 8 Березня, 7-А",(046) 464-32-85,oster_budtex@cg.ukrtel.net,http://okbd.16mb.com,Директор,Бардаченко Тетяна Миколаївна,остерськийколеджбудвництватадизайн,74


In [104]:
univ_data_id[univ_data_id.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)]

,Назва закладу освіти,Код,Код головного закладу,Коротка назва,Назва закладу освіти (англ.),Рік заснування,EDRPOU,Вид закладу освіти,Тип закладу освіти,Форма власності,...,Регіон (юридична),Населений пункт (юридична),Адреса (юридична),Телефон / факс,Електронна пошта,Веб-сайт,Посада керівника,"Прізвище, ім'я, по батькові керівника",id,KOATUU_2020_reg


In [105]:
univ_data_id[(univ_data_id.EDRPOU.str.len() < 8)&(univ_data_id.EDRPOU.str.len().notna())]

,Назва закладу освіти,Код,Код головного закладу,Коротка назва,Назва закладу освіти (англ.),Рік заснування,EDRPOU,Вид закладу освіти,Тип закладу освіти,Форма власності,...,Регіон (юридична),Населений пункт (юридична),Адреса (юридична),Телефон / факс,Електронна пошта,Веб-сайт,Посада керівника,"Прізвище, ім'я, по батькові керівника",id,KOATUU_2020_reg


In [106]:
test_centers_univ = merging(test_centers_mon_final, univ_data_id[['KOATUU_2020_reg', 'id', 'EDRPOU']], ['KOATUU_2020_reg', 'id'])
test_centers_univ

Percentage matched: 37.5 %


,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU
0,"""Запорізький національний університет"" Міністе...",2310136300,23,[2017],запорзькийнацональнийунверситетмнстерстваосвти...,
1,"""Запорізький національний університет"" Міністе...",2310136300,23,[2018],запорзькийнацональнийунверситетмнстерстваосвти...,
2,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2016],навчальновиховнийкомплексовруцькагмназяменандр...,
3,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2017],навчальновиховнийкомплексовруцькагмназяменандр...,
4,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2018],навчальновиховнийкомплексовруцькагмназяменандр...,
...,...,...,...,...,...,...
3476,Ясінянська загальноосвітня школа І-ІІІ ступені...,2123656200,21,"[2016, 2017]",яснянськазагальноосвтняшкола3ступ2рахвськазака...,
3477,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,[2021],загальноосвтняшкола93ступмзмалодеськаобл,
3478,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,"[2017, 2018, 2019, 2020]",загальноосвтняшкола93ступмзмалодеськаобл,
3479,"комунальний заклад освіти ""Середня загальноосв...",1210137800,12,[2021],середнязагальноосвтняшкола62,


# Youcontrol

In [107]:
youcontrol =  pd.read_excel('./school_data/Youcontrol_historical_name.xlsx', dtype='str')
youcontrol 

,Код ЄДРПОУ,Назва,Коротка назва
0,00121146,"ВІДКРИТЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...",NaN
1,00121146,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ"""
2,00121146,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ"""
3,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",NaN
4,00131446,ІНШІ ОРГАНІЗАЦІЙНО-ПРАВОВІ ФОРМИ КОЛЕКТИВНЕ ПІ...,NaN
...,...,...,...
80921,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ"""
80923,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ"""
80924,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ"""


In [108]:
youcontrol.columns

Index(['Код ЄДРПОУ', 'Назва', 'Коротка назва'], dtype='object')

## long name

In [110]:
youcontrol[youcontrol.duplicated(subset=['Назва', 'Код ЄДРПОУ'], keep=False)]

,Код ЄДРПОУ,Назва,Коротка назва
3,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",NaN
5,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",КП МК-26
119,00452713,"ДЕРЖАВНИЙ НАВЧАЛЬНИЙ ЗАКЛАД ""ЗВЕНИГОРОДСЬКИЙ Ц...","ДНЗ ""ЗВЕНИГОРОДСЬКИЙ ЦППРК"""
121,00452713,"ДЕРЖАВНИЙ НАВЧАЛЬНИЙ ЗАКЛАД ""ЗВЕНИГОРОДСЬКИЙ Ц...","ДНЗ ""ЗВЕНИГОРОДСЬКИЙ ЦППРК"""
144,00854423,"КООПЕРАТИВ ""НОВОКРОПИВНИЦЬКИЙ""",NaN
...,...,...,...
80918,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО
80919,60043961,ДЕРЕБЧИНСЬКА СЕРЕДНЯ ЗАГАЛЬНООСВІТНЯ ШКОЛА I-I...,ДЕРЕБЧИНСЬКА СЗШ I-III СТУПЕНІВ
80921,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ"""


In [111]:
youcontrol_long = youcontrol.drop_duplicates(subset=['Назва', 'Код ЄДРПОУ'], keep='first')

In [112]:
create_id(youcontrol_long, 'Назва', 'id')

/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/ZNO-Dataset/notebooks/tables_creation/src/merging_with_id.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:,id] = dataset[attr]
/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/ZNO-Dataset/notebooks/tables_creation/src/merging_with_id.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['id'] = dataset['id'].apply(del_rayon)
/Users/scipyguru/Library/Mobile Documents/com~apple~Cloud

,Код ЄДРПОУ,Назва,Коротка назва,id
0,00121146,"ВІДКРИТЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...",NaN,вдкритеакцонернетовариствопвдентеплоенергомонта
1,00121146,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",публчнеакцонернетовариствопвдентеплоенергомонта
2,00121146,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",приватнеакцонернетовариствопвдентеплоенергомонта
3,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",NaN,колективнепдпримствоблгородднстровськамеханзов...
4,00131446,ІНШІ ОРГАНІЗАЦІЙНО-ПРАВОВІ ФОРМИ КОЛЕКТИВНЕ ПІ...,NaN,колективнепдпримствоблгородднстровськамеханзов...
...,...,...,...,...
80918,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,деребчинськийопорнийзакладзагальнасередньаосвт...
80920,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,деребчинськийопорнийзакладзагальнасередньаосвт...
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинськийзакладзагальнасередн...
80923,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинськийзакладзагальнасередн...


In [113]:
youcontrol_long.rename(columns={'Код ЄДРПОУ':'EDRPOU'}, inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_54084/2371828141.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youcontrol_long.rename(columns={'Код ЄДРПОУ':'EDRPOU'}, inplace=True)


In [114]:
youcontrol_long[(youcontrol_long.EDRPOU.str.len() < 8)&(youcontrol_long.EDRPOU.str.len().notna())]

,EDRPOU,Назва,Коротка назва,id


In [115]:
youcontrol_long[youcontrol_long.duplicated(subset=['id', 'EDRPOU'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
3,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",NaN,колективнепдпримствоблгородднстровськамеханзов...
4,00131446,ІНШІ ОРГАНІЗАЦІЙНО-ПРАВОВІ ФОРМИ КОЛЕКТИВНЕ ПІ...,NaN,колективнепдпримствоблгородднстровськамеханзов...
48,00192471,"КОЛЕКТИВНЕ СПЕЦІАЛІЗОВАНЕ ПІДПРИЄМСТВО ""УКРЕНЕ...",NaN,колективнеспецалзованепдпримствоукренергочормет2
49,00192471,ІНШІ ОРГАНІЗАЦІЙНО-ПРАВОВІ ФОРМИ КОЛЕКТИВНЕ СП...,NaN,колективнеспецалзованепдпримствоукренергочормет2
67,00290280,"ВІДКРИТЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""НОВООЛЕКСАНДРІ...",NaN,вдкритеакцонернетовариствоновоолександрвськийц...
...,...,...,...,...
80917,60043961,ДЕРЕБЧИНСЬКА СЕРЕДНЯ ЗАГАЛЬНООСВІТНЯ ШКОЛА I-I...,ДЕРЕБЧИНСЬКА СЗШ I-III СТУПЕНІВ,деребчинськасереднязагальноосвтняшкола3ступвни...
80918,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,деребчинськийопорнийзакладзагальнасередньаосвт...
80920,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,деребчинськийопорнийзакладзагальнасередньаосвт...
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинськийзакладзагальнасередн...


In [116]:
youcontrol_long = youcontrol_long.drop_duplicates(subset=['id', 'EDRPOU'], keep='first')

In [117]:
youcontrol_long[youcontrol_long.duplicated(subset=['id'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
221,01284979,МІЖШКІЛЬНИЙ НАВЧАЛЬНО-ВИРОБНИЧИЙ КОМБІНАТ,МНВК,мжшкльнийнавчальновиробничийкомбна
494,02218789,"КОМУНАЛЬНА ОРГАНІЗАЦІЯ (УСТАНОВА, ЗАКЛАД) МУЗИ...",NaN,музичнашкола2
498,02218884,"КОМУНАЛЬНА ОРГАНІЗАЦІЯ (УСТАНОВА, ЗАКЛАД) КОМУ...",NaN,музичнашкола3
503,02218921,"КОМУНАЛЬНА ОРГАНІЗАЦІЯ (УСТАНОВА, ЗАКЛАД) КОМУ...",NaN,музичнашкола2
719,02542129,МИКОЛАЇВСЬКИЙ ПРОФЕСІЙНИЙ ЛІЦЕЙ,NaN,миколавськийпрофесйнийлце
...,...,...,...,...
80889,45117367,"ФЕРМЕРСЬКЕ ГОСПОДАРСТВО ""ДОВІРА Б""","ФГ ""ДОВІРА Б""",фермерськегосподарстводовра
80903,45251511,Юріївська гімназія з початковою школою та дошк...,Юріївська гімназія,юрвськагмназязпочатковоюшколоютадошкльнимпдроз...
80905,45275530,Юріївська гімназія з початковою школою та дошк...,Юріївська гімназія,юрвськагмназязпочатковоюшколоютадошкльнимпдроз...
80907,45287518,Кобзарцівська початкова школа з дошкільним під...,Кобзарцівська початкова школа,кобзарцвськапочатковашколаздошкльнимпдроздломс...


In [118]:
youcontrol_long.drop(youcontrol_long[youcontrol_long.duplicated(subset=['id'], keep=False)].index.values.tolist()  , inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_54084/2842222147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youcontrol_long.drop(youcontrol_long[youcontrol_long.duplicated(subset=['id'], keep=False)].index.values.tolist()  , inplace=True)


In [119]:
test_centers_youcontrol_long = merging(test_centers_univ, youcontrol_long[['id', 'EDRPOU']], ['id'])
test_centers_youcontrol_long

Percentage matched: 81.6 %


,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU
0,"""Запорізький національний університет"" Міністе...",2310136300,23,[2017],запорзькийнацональнийунверситетмнстерстваосвти...,
1,"""Запорізький національний університет"" Міністе...",2310136300,23,[2018],запорзькийнацональнийунверситетмнстерстваосвти...,
2,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2016],навчальновиховнийкомплексовруцькагмназяменандр...,06670569
3,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2017],навчальновиховнийкомплексовруцькагмназяменандр...,06670569
4,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2018],навчальновиховнийкомплексовруцькагмназяменандр...,06670569
...,...,...,...,...,...,...
3476,Ясінянська загальноосвітня школа І-ІІІ ступені...,2123656200,21,"[2016, 2017]",яснянськазагальноосвтняшкола3ступ2рахвськазака...,
3477,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,[2021],загальноосвтняшкола93ступмзмалодеськаобл,22499879
3478,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,"[2017, 2018, 2019, 2020]",загальноосвтняшкола93ступмзмалодеськаобл,22499879
3479,"комунальний заклад освіти ""Середня загальноосв...",1210137800,12,[2021],середнязагальноосвтняшкола62,


## short name


In [120]:
youcontrol_short = youcontrol[youcontrol['Коротка назва'].notna()]

In [121]:
youcontrol_short.rename(columns={'Код ЄДРПОУ':'EDRPOU'}, inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_54084/692976443.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youcontrol_short.rename(columns={'Код ЄДРПОУ':'EDRPOU'}, inplace=True)


In [122]:
create_id(youcontrol_short, 'Коротка назва', 'id')

/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/ZNO-Dataset/notebooks/tables_creation/src/merging_with_id.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:,id] = dataset[attr]
/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/ZNO-Dataset/notebooks/tables_creation/src/merging_with_id.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['id'] = dataset['id'].apply(del_rayon)
/Users/scipyguru/Library/Mobile Documents/com~apple~Cloud

,EDRPOU,Назва,Коротка назва,id
1,00121146,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",атпте
2,00121146,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",атпте
5,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",КП МК-26,кпмк26
7,00131564,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПРИКАРПАТТЯОБЛ...","ПАТ ""ПРИКАРПАТТЯОБЛЕНЕРГО""",патприкарпаттяобленерг
8,00131564,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПРИКАРПАТТЯОБЛ...","АТ ""ПРИКАРПАТТЯОБЛЕНЕРГО""",атприкарпаттяобленерг
...,...,...,...,...
80921,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,доззс
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинськийзакладзагальнасередн...
80923,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинськийзакладзагальнасередн...
80924,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинськийзакладзагальнасередн...


In [123]:
youcontrol_short[youcontrol_short.duplicated(subset=['Коротка назва', 'EDRPOU'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
1,00121146,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",атпте
2,00121146,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",атпте
11,00131954,"АКЦІОНЕРНА ТОВАРИСТВО ""ХАРКІВОБЛЕНЕРГО""","АТ ""ХАРКІВОБЛЕНЕРГО""",атхарквобленерг
12,00131954,"АКЦІОНЕРНЕ ТОВАРИСТВО ""ХАРКІВОБЛЕНЕРГО""","АТ ""ХАРКІВОБЛЕНЕРГО""",атхарквобленерг
62,00220434,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""НАУКОВО-ВИРОБН...","АТ ""НВАТ ""ВНДІКОМПРЕСОРМАШ""",атнватвндкомпресорма
...,...,...,...,...
80920,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,доззс
80921,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,доззс
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинськийзакладзагальнасередн...
80923,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинськийзакладзагальнасередн...


In [124]:
youcontrol_short = youcontrol_short.drop_duplicates(subset=['Коротка назва', 'EDRPOU'], keep='first')

In [125]:
youcontrol_short[youcontrol_short.duplicated(subset=['id', 'EDRPOU'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
103,00389357,ДЕРЖАВНИЙ НАВЧАЛЬНИЙ ЗАКЛАД ЛЬВІВСЬКЕ ВИЩЕ ПРО...,ДНЗ ЛЬВІВСЬКЕ ВПУХТ,льввськевпух
104,00389357,"ДЕРЖАВНИЙ НАВЧАЛЬНИЙ ЗАКЛАД ""ЛЬВІВСЬКЕ ВИЩЕ ПР...","ДНЗ ""ЛЬВІВСЬКЕ ВПУХТ""",льввськевпух
157,00914361,ТЕРНОПІЛЬСЬКИЙ НАВЧАЛЬНО-КУРСОВИЙ КОМБІНАТ,ТЕРНОПІЛЬСЬКИЙ НКК,тернопльськийнк
158,00914361,"ДЕРЖАВНИЙ НАВЧАЛЬНИЙ ЗАКЛАД ""ТЕРНОПІЛЬСЬКИЙ НА...","ДНЗ ""ТЕРНОПІЛЬСЬКИЙ НКК""",тернопльськийнк
174,00951592,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ЄЛИЗАВЕТІВСЬК...","ПРАТ ""ЄЛИЗАВЕТІВСЬКИЙ ЕЛЕВАТОР""",пратлизаветвськийелевато
...,...,...,...,...
80431,43861328,"НАЦІОНАЛЬНИЙ УНІВЕРСИТЕТ ""ОДЕСЬКА ПОЛІТЕХНІКА""",ОДЕСЬКА ПОЛІТЕХНІКА,одеськаполтехнк
80518,44129392,ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ПРИВАТ...,"ТОВ ПЗ ""ХЛ ""ХАНІ АКАДЕМІЯ"" ХО",пзхлханакадемях
80519,44129392,ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ПРИВАТ...,"ТОВ ПЗ ""ХЛ ""ХАНІ АКАДЕМІЯ"""" ХО",пзхлханакадемях
80581,44271671,РУБІЖАНСЬКИЙ ЛІЦЕЙ РУБІЖАНСЬКОЇ МІСЬКОЇ РАДИ Л...,РУБІЖАНСЬКИЙ ЛІЦЕЙ,рубжанськийлце


In [126]:
youcontrol_short = youcontrol_short.drop_duplicates(subset=['id', 'EDRPOU'], keep='first')

In [127]:
youcontrol_short[youcontrol_short.duplicated(subset=['id'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
221,01284979,МІЖШКІЛЬНИЙ НАВЧАЛЬНО-ВИРОБНИЧИЙ КОМБІНАТ,МНВК,мнв
295,01489345,ЛІЦЕЙ №18 ДЕСНЯНСЬКОГО РАЙОНУ МІСТА КИЄВА,ЛІЦЕЙ №18,лцей18
332,01730137,ОЛЕКСАНДРІВСЬКИЙ КОМБІНАТ КООПЕРАТИВНОЇ ПРОМИС...,ОЛЕКСАНДРІВСЬКИЙ ККП,олександрвськийкк
418,02137051,Ліцей с. Стрілки Стрілківської сільської ради ...,Стрілківський ліцей,стрлквськийлце
510,02218967,"КОМУНАЛЬНИЙ ЗАКЛАД ""МИСТЕЦЬКА ШКОЛА №2 КРАМАТО...",МИСТЕЦЬКА ШКОЛА № 2,мистецькашкола2
...,...,...,...,...
80905,45275530,Юріївська гімназія з початковою школою та дошк...,Юріївська гімназія,юрвськагмназ
80906,45277868,"Комунальний заклад ""Петрівська гімназія"" Кодим...","КЗ ""Петрівська гімназія""",петрвськагмназ
80907,45287518,Кобзарцівська початкова школа з дошкільним під...,Кобзарцівська початкова школа,кобзарцвськапочатковашкол
80912,45383386,Кобзарцівська початкова школа з дошкільним під...,Кобзарцівська початкова школа,кобзарцвськапочатковашкол


In [128]:
youcontrol_short.drop(youcontrol_short[youcontrol_short.duplicated(subset=['id'], keep=False)].index.values.tolist()  , inplace=True)

In [130]:
test_centers_youcontrol_short = merging(test_centers_youcontrol_long, youcontrol_short[['id', 'EDRPOU']], ['id'])
test_centers_youcontrol_short 

Percentage matched: 82.2 %


,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU
0,"""Запорізький національний університет"" Міністе...",2310136300,23,[2017],запорзькийнацональнийунверситетмнстерстваосвти...,
1,"""Запорізький національний університет"" Міністе...",2310136300,23,[2018],запорзькийнацональнийунверситетмнстерстваосвти...,
2,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2016],навчальновиховнийкомплексовруцькагмназяменандр...,06670569
3,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2017],навчальновиховнийкомплексовруцькагмназяменандр...,06670569
4,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2018],навчальновиховнийкомплексовруцькагмназяменандр...,06670569
...,...,...,...,...,...,...
3476,Ясінянська загальноосвітня школа І-ІІІ ступені...,2123656200,21,"[2016, 2017]",яснянськазагальноосвтняшкола3ступ2рахвськазака...,
3477,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,[2021],загальноосвтняшкола93ступмзмалодеськаобл,22499879
3478,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,"[2017, 2018, 2019, 2020]",загальноосвтняшкола93ступмзмалодеськаобл,22499879
3479,"комунальний заклад освіти ""Середня загальноосв...",1210137800,12,[2021],середнязагальноосвтняшкола62,22336611


# From schools

In [131]:
schools_data = pd.read_csv('./schools_edrpou.csv',  dtype = str)
schools_data

,eoname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU
0,"Вiдокремлений структурний пiдроздiл ""Краматорс...",0510100000,05,[2023],краматорськийколеджпромисловостнформацйнихтехн...,04601943
1,"ВСП ""Технологічно-промисловий фаховий коледж В...",0510100000,05,[2022],технологчнопромисловийколеджвницькогонацональн...,00419667
2,Вище професійне училище №11 м. Вінниці,0510100000,05,"[2018, 2019, 2020, 2021, 2022, 2023]",вищепрофесйнеучилище11мвниц,03065891
3,Вище художнє професійно-технічне училище № 5 м...,0510100000,05,[2023],вищехудожнпрофесйнотехнчнеучилище5мвниц,02539890
4,Вище художнє професійно-технічне училище №5 м....,0510100000,05,"[2018, 2019, 2020, 2021, 2022]",вищехудожнпрофесйнотехнчнеучилище5мвниц,02539890
...,...,...,...,...,...,...
32052,Школа №25 І-ІІІ ступенів Шевченківського район...,8039100000,80,"[2016, 2017, 2018, 2019, 2020, 2021]",школа253ступмкив,22880786
32053,Школа №27 І-ІІІ ступенів Шевченківського район...,8039100000,80,[2016],школа273ступмкив,26125710
32054,Школа №70 І-ІІІ ступенів Шевченківського район...,8039100000,80,[2016],школа703ступмкив,22881828
32055,Школа №95 І-ІІІ ступенів Шевченківського район...,8039100000,80,[2017],школа953ступмкив,26125905


In [133]:
schools_data.columns

Index(['eoname', 'KOATUU_2020', 'KOATUU_2020_reg', 'year', 'id', 'EDRPOU'], dtype='object')

In [134]:
schools_data = schools_data[['eoname', 'KOATUU_2020', 'EDRPOU', 'KOATUU_2020_reg']]

In [135]:
schools_data['temp_eoname'] = schools_data['eoname'].str.replace(r'\r\n|\n', '', regex=True).str.strip()

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_54084/2721840567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schools_data['temp_eoname'] = schools_data['eoname'].str.replace(r'\r\n|\n', '', regex=True).str.strip()


In [136]:
schools_data[schools_data.duplicated(subset=['temp_eoname', 'EDRPOU', 'KOATUU_2020_reg'], keep=False)]

,eoname,KOATUU_2020,EDRPOU,KOATUU_2020_reg,temp_eoname
323,Войнашівська загальноосвітня школа I-III ступе...,0520280601,13326973,05,Войнашівська загальноосвітня школа I-III ступе...
326,Войнашівська загальноосвітня школа I-III ступе...,0520280603,13326973,05,Войнашівська загальноосвітня школа I-III ступе...
483,Степенівський навчально - виховний комплекс: з...,0520687603,23562485,05,Степенівський навчально - виховний комплекс: з...
484,Степенівський навчально - виховний комплекс: з...,0520687603,23562485,05,Степенівський навчально - виховний комплекс: з...
619,"Комунальний заклад ""Сербинівський навчально-ви...",0521085403,39977763,05,"Комунальний заклад ""Сербинівський навчально-ви..."
...,...,...,...,...,...
31714,"Приватний заклад передвищої освіти ""Мистецький...",8038500000,30728300,80,"Приватний заклад передвищої освіти ""Мистецький..."
31719,"Товариство з обмеженою відповідальністю ""Колег...",8038500000,33635570,80,"Товариство з обмеженою відповідальністю ""Колег..."
31828,Відокремлений структурний підрозділ Київський ...,8038900000,39867890,80,Відокремлений структурний підрозділ Київський ...
31936,Відокремлений структурний підрозділ Київський ...,8039100000,39867890,80,Відокремлений структурний підрозділ Київський ...


In [137]:
schools_data = schools_data.drop_duplicates(subset=['temp_eoname', 'EDRPOU', 'KOATUU_2020_reg'], keep='first')

In [138]:
schools_data[schools_data.duplicated(subset=['temp_eoname', 'KOATUU_2020_reg'], keep=False)]

,eoname,KOATUU_2020,EDRPOU,KOATUU_2020_reg,temp_eoname


In [139]:
schools_data = schools_data.drop_duplicates(subset=['temp_eoname', 'KOATUU_2020_reg'], keep='first')

In [141]:
test_centers_youcontrol_short['temp_eoname'] = test_centers_youcontrol_short['ptname'].str.replace(r'\r\n|\n', '', regex=True).str.strip()

In [142]:
test_centers_final = merging(test_centers_youcontrol_short, schools_data[['temp_eoname', 'EDRPOU', 'KOATUU_2020_reg']], ['temp_eoname', 'KOATUU_2020_reg'])
test_centers_final

Percentage matched: 95.5 %


,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU,temp_eoname
0,"""Запорізький національний університет"" Міністе...",2310136300,23,[2017],запорзькийнацональнийунверситетмнстерстваосвти...,,"""Запорізький національний університет"" Міністе..."
1,"""Запорізький національний університет"" Міністе...",2310136300,23,[2018],запорзькийнацональнийунверситетмнстерстваосвти...,,"""Запорізький національний університет"" Міністе..."
2,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2016],навчальновиховнийкомплексовруцькагмназяменандр...,06670569,"""Навчально-виховний комплекс ""Овруцька гімназі..."
3,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2017],навчальновиховнийкомплексовруцькагмназяменандр...,06670569,"""Навчально-виховний комплекс ""Овруцька гімназі..."
4,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2018],навчальновиховнийкомплексовруцькагмназяменандр...,06670569,"""Навчально-виховний комплекс ""Овруцька гімназі..."
...,...,...,...,...,...,...,...
3476,Ясінянська загальноосвітня школа І-ІІІ ступені...,2123656200,21,"[2016, 2017]",яснянськазагальноосвтняшкола3ступ2рахвськазака...,26326118,Ясінянська загальноосвітня школа І-ІІІ ступені...
3477,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,[2021],загальноосвтняшкола93ступмзмалодеськаобл,22499879,загальноосвітня школа №9 І – ІІІ ступенів міст...
3478,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,"[2017, 2018, 2019, 2020]",загальноосвтняшкола93ступмзмалодеськаобл,22499879,загальноосвітня школа №9 І – ІІІ ступенів міст...
3479,"комунальний заклад освіти ""Середня загальноосв...",1210137800,12,[2021],середнязагальноосвтняшкола62,22336611,"комунальний заклад освіти ""Середня загальноосв..."


In [143]:
test_centers_final[test_centers_final.EDRPOU == '']

,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU,temp_eoname
0,"""Запорізький національний університет"" Міністе...",2310136300,23,[2017],запорзькийнацональнийунверситетмнстерстваосвти...,,"""Запорізький національний університет"" Міністе..."
1,"""Запорізький національний університет"" Міністе...",2310136300,23,[2018],запорзькийнацональнийунверситетмнстерстваосвти...,,"""Запорізький національний університет"" Міністе..."
50,Інженерний інститут Запорізького національного...,2310136900,23,[2019],нженернийнститутзапорзькогонацональногоунверситет,,Інженерний інститут Запорізького національного...
51,Інститут післядипломної педагогічної освіти Ки...,8036600000,80,[2016],нститутпслядипломнапедагогчнаосвтикивськогоунв...,,Інститут післядипломної педагогічної освіти Ки...
66,Аграрний коледж управління і права Полтавської...,5310137000,53,"[2016, 2017]",аграрнийколеджуправлняправаполтавськадержавнаа...,,Аграрний коледж управління і права Полтавської...
...,...,...,...,...,...,...,...
3311,"Центральноукраїнський інститут ПрАТ ""ВНЗ"" ""МАУП""",3510136600,35,[2018],центральноукранськийнститутпратвнзмау,,"Центральноукраїнський інститут ПрАТ ""ВНЗ"" ""МАУП"""
3312,Центральноукраїнський інститут Приватне акціон...,3510136600,35,"[2019, 2020]",центральноукранськийнститутприватнеакцонернето...,,Центральноукраїнський інститут Приватне акціон...
3337,"Черкаський навчально-науковий інститут ДВНЗ ""У...",7110136400,71,"[2016, 2017, 2018]",черкаськийнавчальнонауковийнститутдвнзунверсит...,,"Черкаський навчально-науковий інститут ДВНЗ ""У..."
3371,Чернігівський національний педагогічний універ...,7410136300,74,"[2016, 2017, 2018]",чернгвськийнацональнийпедагогчнийунверситетмен...,,Чернігівський національний педагогічний універ...


In [144]:
test_centers_final.drop(columns=['temp_eoname'], inplace=True)

In [146]:
test_centers_final = test_centers_final.sort_values(by=['KOATUU_2020', 'ptname'])

In [147]:
test_centers_final.to_csv('test_centers_edrpou.csv', index = False)

# by hand

In [148]:
data_hand = pd.read_csv('./school_data/testcenters_by_hand.csv',  dtype = str)
data_hand

,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU
0,Вище професійне училище №11 м. Вінниці,0510100000,05,"[2018, 2019, 2020, 2021]",вищепрофесйнеучилище11мвниц,03065891
1,"Відокремлений структурний підрозділ ""Вінницьки...",0510100000,05,[2021],вницькийколеджнацональногоунверситетухарчовихт...,35725833
2,Відокремлений структурний підрозділ Вінницький...,0510100000,05,[2021],вницькийколеджбудвництваархтектуритадизайнукив...,38814461
3,Вінницький коледж Національного університету х...,0510100000,05,"[2018, 2019, 2020]",вницькийколеджнацональногоунверситетухарчовихт...,35725833
4,Вінницький коледж будівництва і архітектури Ки...,0510100000,05,"[2016, 2017, 2018, 2019, 2020]",вницькийколеджбудвництваархтектурикивськогонац...,38814461
...,...,...,...,...,...,...
3476,Школа №199 І-ІІІ ступенів Шевченківського райо...,8039100000,80,[2016],школа1993ступмкив,22881923
3477,Школа №25 І-ІІІ ступенів Шевченківського район...,8039100000,80,"[2018, 2019, 2020, 2021]",школа253ступмкив,22880786
3478,Школа №70 І-ІІІ ступенів Шевченківського район...,8039100000,80,[2016],школа703ступмкив,22881828
3479,Школа №95 І-ІІІ ступенів Шевченківського район...,8039100000,80,[2017],школа953ступмкив,26125905


In [149]:
data_hand.columns

Index(['ptname', 'KOATUU_2020', 'KOATUU_2020_reg', 'year', 'id', 'EDRPOU'], dtype='object')

In [150]:
data_hand = data_hand[['ptname', 'KOATUU_2020', 'EDRPOU', 'KOATUU_2020_reg']]

In [151]:
data_hand['temp_eoname'] = data_hand['ptname'].str.replace(r'\r\n|\n', '', regex=True).str.strip()

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_54084/652156521.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_hand['temp_eoname'] = data_hand['ptname'].str.replace(r'\r\n|\n', '', regex=True).str.strip()


In [152]:
data_hand[data_hand.duplicated(subset=['temp_eoname', 'EDRPOU', 'KOATUU_2020_reg'], keep=False)]

,ptname,KOATUU_2020,EDRPOU,KOATUU_2020_reg,temp_eoname
162,"Луцький навчально-виховний комплекс ""Загальноо...",0710100000,26276366,07,"Луцький навчально-виховний комплекс ""Загальноо..."
163,"Луцький навчально-виховний комплекс ""Загальноо...",0710100000,26276366,07,"Луцький навчально-виховний комплекс ""Загальноо..."
196,Нововолинська загальноосвітня школа І-ІІІ ступ...,0710700000,20144391,07,Нововолинська загальноосвітня школа І-ІІІ ступ...
197,Нововолинська загальноосвітня школа І-ІІІ ступ...,0710700000,20144391,07,Нововолинська загальноосвітня школа І-ІІІ ступ...
272,"Вищий навчальний приватний заклад ""Дніпровськи...",1210136600,32495593,12,"Вищий навчальний приватний заклад ""Дніпровськи..."
...,...,...,...,...,...
2741,Чугуївська загальноосвітня школа I-III ступені...,6312000000,24335030,63,Чугуївська загальноосвітня школа I-III ступені...
2779,Херсонський політехнічний коледж Одеського нац...,6510136300,00237191,65,Херсонський політехнічний коледж Одеського нац...
2800,Херсонський політехнічний коледж Одеського нац...,6510136900,00237191,65,Херсонський політехнічний коледж Одеського нац...
2942,"Філія ""Черкаський кооперативний економіко-прав...",7110136400,39843664,71,"Філія ""Черкаський кооперативний економіко-прав..."


In [153]:
data_hand = data_hand.drop_duplicates(subset=['temp_eoname', 'EDRPOU', 'KOATUU_2020_reg'], keep='first')

In [154]:
data_hand[data_hand.duplicated(subset=['temp_eoname', 'KOATUU_2020_reg'], keep=False)]

,ptname,KOATUU_2020,EDRPOU,KOATUU_2020_reg,temp_eoname


In [156]:
test_centers_final['temp_eoname'] = test_centers_final['ptname'].str.replace(r'\r\n|\n', '', regex=True).str.strip()

In [157]:
test_centers = merging(test_centers_final, data_hand[['temp_eoname', 'EDRPOU', 'KOATUU_2020_reg']], ['temp_eoname', 'KOATUU_2020_reg'])
test_centers

Percentage matched: 100.0 %


,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU,temp_eoname
0,Вище професійне училище №11 м. Вінниці,0510100000,05,"[2018, 2019, 2020, 2021]",вищепрофесйнеучилище11мвниц,03065891,Вище професійне училище №11 м. Вінниці
1,"Відокремлений структурний підрозділ ""Вінницьки...",0510100000,05,[2021],вницькийколеджнацональногоунверситетухарчовихт...,35725833,"Відокремлений структурний підрозділ ""Вінницьки..."
2,Відокремлений структурний підрозділ Вінницький...,0510100000,05,[2021],вницькийколеджбудвництваархтектуритадизайнукив...,38814461,Відокремлений структурний підрозділ Вінницький...
3,Вінницький коледж Національного університету х...,0510100000,05,"[2018, 2019, 2020]",вницькийколеджнацональногоунверситетухарчовихт...,35725833,Вінницький коледж Національного університету х...
4,Вінницький коледж будівництва і архітектури Ки...,0510100000,05,"[2016, 2017, 2018, 2019, 2020]",вницькийколеджбудвництваархтектурикивськогонац...,38814461,Вінницький коледж будівництва і архітектури Ки...
...,...,...,...,...,...,...,...
3476,Школа №199 І-ІІІ ступенів Шевченківського райо...,8039100000,80,[2016],школа1993ступмкив,22881923,Школа №199 І-ІІІ ступенів Шевченківського райо...
3477,Школа №25 І-ІІІ ступенів Шевченківського район...,8039100000,80,"[2018, 2019, 2020, 2021]",школа253ступмкив,22880786,Школа №25 І-ІІІ ступенів Шевченківського район...
3478,Школа №70 І-ІІІ ступенів Шевченківського район...,8039100000,80,[2016],школа703ступмкив,22881828,Школа №70 І-ІІІ ступенів Шевченківського район...
3479,Школа №95 І-ІІІ ступенів Шевченківського район...,8039100000,80,[2017],школа953ступмкив,26125905,Школа №95 І-ІІІ ступенів Шевченківського район...


In [158]:
test_centers.drop(columns=['temp_eoname'], inplace=True)

In [160]:
test_centers = test_centers.sort_values(by=['KOATUU_2020', 'ptname'])

In [161]:
test_centers.to_csv('testcenters_edrpou.csv', index = False)

# Check

In [162]:
result_df = test_centers.groupby('EDRPOU')['year'].apply(lambda x: sorted(y for sublist in x for y in sublist)).reset_index()

result_df.columns

Index(['EDRPOU', 'year'], dtype='object')

In [163]:
duplicates_mask = result_df['year'].apply(lambda x: len(x) != len(set(x)))

# Filter rows with duplicates
duplicates_df = result_df[duplicates_mask]

duplicates_df 

,EDRPOU,year
13,00208752,"[2016, 2016, 2017, 2018, 2019, 2020, 2021]"
33,00493787,"[2016, 2016, 2017, 2018, 2019, 2020, 2021]"
68,01566117,"[2016, 2016, 2017, 2017, 2018, 2018]"
78,01597997,"[2016, 2017, 2018, 2019, 2020, 2021, 2021]"
97,02066747,"[2021, 2021]"
...,...,...
2022,37664469,"[2016, 2016, 2017, 2018, 2019, 2020, 2021]"
2046,39568620,"[2016, 2017, 2018, 2019, 2020, 2021, 2021]"
2084,42415315,"[2019, 2020, 2021, 2021, 2021]"
2087,42664633,"[2020, 2020, 2021]"


In [164]:
duplicates_df.EDRPOU.unique()

array(['00208752', '00493787', '01566117', '01597997', '02066747',
       '02066769', '02071062', '02071100', '02071180', '02125243',
       '02125510', '02125639', '02128371', '02139630', '02541409',
       '05408289', '05460798', '05536076', '08751177', '14027439',
       '14163438', '20103727', '20132672', '20523929', '20553155',
       '20570389', '20933314', '21065765', '21151237', '21161210',
       '21431276', '21437505', '21438114', '21726793', '22055214',
       '22512781', '22545183', '22548394', '22664386', '22798702',
       '22876023', '23210450', '23244093', '23541721', '23545802',
       '23545860', '23625562', '23681349', '23802771', '23806540',
       '23863488', '23887299', '24135555', '24193601', '24211658',
       '24215662', '24357173', '24414670', '24559947', '24622284',
       '24626968', '24630817', '24751219', '24882690', '24892121',
       '24948577', '24952840', '25294636', '25300306', '25322733',
       '25454328', '25645011', '25689356', '25706014', '258114

In [165]:
problems_mask = test_centers['EDRPOU'].apply(lambda x: x in duplicates_df.EDRPOU.unique())

# Filter rows with duplicates
problems = test_centers[problems_mask]

problems

,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU
91,Ліцей № 7 м. Гайсин Вінницької області,0520810100,05,"[2019, 2021]",лцей7мгайсинвницькаобл,21726793
92,Ліцей № 7 м. Гайсин Вінницької області (КОРПУС 1),0520810100,05,[2020],лцей7мгайсинвницькаобл,21726793
93,Ліцей № 7 м. Гайсин Вінницької області (КОРПУС 2),0520810100,05,[2020],лцей7мгайсинвницькаобл,21726793
94,Ліцей № 7 м. Гайсин Вінницької області (Корпус 1),0520810100,05,[2021],лцей7мгайсинвницькаобл,21726793
95,Ліцей № 7 м. Гайсин Вінницької області (ПТ №1),0520810100,05,[2021],лцей7мгайсинвницькаобл,21726793
...,...,...,...,...,...,...
3313,Школа І-ІІІ ступенів №29,8038000000,80,"[2020, 2021]",школа3ступ29,23887299
3399,Національна академія внутрішніх справ (Сектор 1),8038900000,80,[2021],нацональнаакадемявнутршнхспра,08751177
3400,Національна академія внутрішніх справ (Сектор 2),8038900000,80,[2021],нацональнаакадемявнутршнхспра,08751177
3401,Національна академія внутрішніх справ (Сектор 3),8038900000,80,[2021],нацональнаакадемявнутршнхспра,08751177


In [166]:
problems.to_csv('test_centers_problems.csv', index = False)